# Speed Skydiving scoring bulk files processor
Uses the `./data` directory as a data lake.  It finds all valid FlySight files in that directory, from 0 to as many as it finds and fit in memory, and process them all.  Summary of "all files" or "all jumps" at the bottom of the document.

In [37]:
from collections import namedtuple
from copy import deepcopy

from ssscoring import BREAKOFF_ALTITUDE
from ssscoring import FT_IN_M
from ssscoring import PERFORMANCE_WINDOW_LENGTH
from ssscoring import aggregateResults
from ssscoring import convertFlySight2SSScoring
from ssscoring import dropNonSkydiveDataFrom
from ssscoring import getAllSpeedJumpFilesFrom
from ssscoring import getSpeedSkydiveFrom
from ssscoring import isValidJump
from ssscoring import isValidMinimumAltitude
from ssscoring import jumpAnalysisTable
from ssscoring import processAllJumpFiles
from ssscoring import roundedAggregateResults
from ssscoring import totalResultsFrom
from ssscoring import validFlySightHeaderIn
from ssscoring.notebook import SPEED_COLORS
from ssscoring.notebook import graphAltitude
from ssscoring.notebook import graphAngle
from ssscoring.notebook import graphJumpResult
from ssscoring.notebook import initializeExtraYRanges
from ssscoring.notebook import initializePlot

import csv
import os
import os.path as path

import bokeh.plotting as bp
import bokeh.models as bm
import ipywidgets as widgets
import pandas as pd

In [38]:
DATA_LAKE_ROOT = './data'

In [39]:
jumpFiles = getAllSpeedJumpFilesFrom(DATA_LAKE_ROOT)

In [40]:
jumpResults = processAllJumpFiles(jumpFiles)

---
## <span style="color: yellow">IMPORTANT</span> - Set the drop zone altitude MSL

Set the value in ft (even though SSScoring uses meters) out of convenience because all DZs in the US, many in Asia and Mexico, and many in Europe express the altitude in feet.

### Some DZs

| Drop zone | Alt (ft) |
|-----------|----------|
| Bay Area Skydiving | 23 |
| Skydive Arizona | 1509 |
| Thai Sky Adventures | 21 |

In [41]:
dropZoneAltMSL = 92.00

In [42]:
dropZoneAltMSLMeters = dropZoneAltMSL/FT_IN_M
display(widgets.HTML('<h2>DZ Altitude = <span style = "color: green">%7.2f ft</span> (%7.2f m)<h1>' % (dropZoneAltMSL, dropZoneAltMSLMeters)))

HTML(value='<h2>DZ Altitude = <span style = "color: green">  92.00 ft</span> (  28.04 m)<h1>')

---
## Results

In [43]:
aggregate = aggregateResults(jumpResults)
sumResults = totalResultsFrom(aggregate)
display(aggregate)

,score,5.0,10.0,15.0,20.0,25.0,finalTime,maxSpeed
01-00-TJ-BASELINE,479.736,175.716,315.360,413.568,466.956,483.552,24.8,483.552
10-53-19,458.496,170.892,284.976,387.324,434.520,458.964,25.0,462.708
11-38-43,457.344,170.316,301.140,385.236,435.672,460.188,25.0,461.736
12-55-39,472.446,177.048,295.560,397.872,457.092,475.164,25.0,476.712
13-45-34,460.134,179.640,304.488,383.724,441.288,463.752,25.0,464.544


### Rounded results for training log

In [44]:
roundedResults = roundedAggregateResults(jumpResults)

### All jumps

In [45]:
def dumpSimple(table: pd.DataFrame):
    fileName = os.path.join(DATA_LAKE_ROOT, resultRef+'.tsv')
    simpleTable = table.copy()
    simpleTable = simpleTable.drop('speedAngle', axis = 1)
    simpleTable.to_csv(fileName, sep = '\t', index = False)

In [46]:
def displayJumpDataIn(resultsTable: pd.DataFrame):
    table = resultsTable.copy()
    # Experimental
    # For more information on the `interpolate` method and its options, see the [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.interpolate.html).
    # Additionally, you can also consider using other libraries like `scipy` which provides more advanced interpolation methods, such as `interp1d` or `griddata`. See the [scipy documentation](https://docs.scipy.org/doc/scipy/reference/interpolate.html) for more information.
    table.vKMh = table.vKMh.apply(round)
    table.hKMh = table.hKMh.apply(round)
    table['altitude (ft)'] = table['altitude (ft)'].apply(round)
    table.netVectorKMh = table.netVectorKMh.apply(round)
    table.index = ['']*len(table)
    display(table)
    # dumpSimple(table)

In [47]:
allJumpsPlot = initializePlot('All jumps in set')
jumpNumber = 0
mixColor = 0
for resultRef in sorted(jumpResults.keys()):
    jumpResult = jumpResults[resultRef]
    if jumpResult.score > 0.0:
        validJumpStatus = '<hr><h1><span style="color: %s">%s jump - %s - score = %.02f km/h</span></h1>' % (jumpResult.color, resultRef, jumpResult.result, jumpResult.score)        
    else:
        validJumpStatus = '<hr><h1><span style="color: %s">%s jump - %s</span></h1>' % (jumpResult.color, resultRef, jumpResult.result)
    display(widgets.HTML(validJumpStatus))

    maxSpeed = jumpResult.maxSpeed
    window = jumpResult.window
    mixColor = (mixColor+1)%len(SPEED_COLORS)
    if jumpResult.score > 0.0:
        display(widgets.HTML('<h3>Max speed = {0:,.0f}; '.format(maxSpeed)+('exit at %d m (%d ft), end scoring window at %d m (%d ft)'%(window.start, 3.2808*window.start, window.end, 3.2808*window.end))))
        if not isValidMinimumAltitude(jumpResult.data.altitudeASL.max()):
            display(widgets.HTML('<span style="color: yellow"><span style="font-weight: bold">Warning:</span> exit altitude ASL was lower than the minimum scoring altitude according to IPC and USPA.'))
        display(widgets.HTML('</h3>'))
        # TODO: Move this to an in-notebook function or module.
        displayJumpDataIn(jumpResult.table)
        individualPlot = initializePlot(resultRef)
        individualPlot = initializeExtraYRanges(individualPlot, startY = min(jumpResult.data.altitudeASLFt)-500.0, endY = max(jumpResult.data.altitudeASLFt)+500.0)
        graphAltitude(individualPlot, jumpResult)
        graphAngle(individualPlot, jumpResult)
        hoverValue = bm.HoverTool(tooltips = [ ('Y-val', '@y{0.00}'), ])
        individualPlot.add_tools(hoverValue)
        graphJumpResult(individualPlot,
                        jumpResult,
                        lineColor = SPEED_COLORS[0])
        graphJumpResult(allJumpsPlot,
                        jumpResult,
                        lineColor = SPEED_COLORS[mixColor],
                        legend = '%s - %.2f' % (resultRef, jumpResult.score),
                        showIt = False)

HTML(value='<hr><h1><span style="color: #0f0">01-00-TJ-BASELINE jump - 🟢 valid - score = 479.74 km/h</span></h…

HTML(value='<h3>Max speed = 484; exit at 4212 m (13818 ft), end scoring window at 1956 m (6417 ft)')

HTML(value='</h3>')

,time,vKMh,hKMh,speedAngle,netVectorKMh,altitude (ft)
,5.0,176,154,48.8,234,13389
,10.0,315,113,70.2,335,12241
,15.0,414,87,78.2,423,10556
,20.0,467,68,81.7,472,8533
,25.0,484,50,84.0,486,6440


HTML(value='<hr><h1><span style="color: #0f0">10-53-19 jump - 🟢 valid - score = 458.50 km/h</span></h1>')

...

,time,vKMh,hKMh,speedAngle,netVectorKMh,altitude (ft)
,5.0,177,78,66.2,193,13498
,10.0,296,68,77.0,303,12406
,15.0,398,78,78.8,406,10814
,20.0,457,49,83.9,460,8846
,25.0,475,32,86.1,476,6709


HTML(value='<hr><h1><span style="color: #0f0">13-45-34 jump - 🟢 valid - score = 460.13 km/h</span></h1>')

HTML(value='<h3>Max speed = 465; exit at 4305 m (14124 ft), end scoring window at 2049 m (6722 ft)')

HTML(value='</h3>')

,time,vKMh,hKMh,speedAngle,netVectorKMh,altitude (ft)
,5.0,180,79,66.2,196,13679
,10.0,304,31,84.2,306,12542
,15.0,384,29,85.7,385,11000
,20.0,441,22,87.2,442,9101
,25.0,464,30,86.3,465,7027


---
## All speed skydives in data lake summary

In [48]:
display(roundedResults)
display(sumResults)
bp.show(allJumpsPlot)

,score,5.0,10.0,15.0,20.0,25.0,finalTime,maxSpeed
01-00-TJ-BASELINE,480,176,315,414,467,484,25.0,484
10-53-19,458,171,285,387,435,459,25.0,463
11-38-43,457,170,301,385,436,460,25.0,462
12-55-39,472,177,296,398,457,475,25.0,477
13-45-34,460,180,304,384,441,464,25.0,465


,totalSpeed,meanSpeed,maxScore
totalSpeed,2328.156,465.6312,479.736
